In [1]:
import requests
import numpy as np
import pandas as pd
import datetime as dt


# Set globals
API_KEY = 'fbf2a3cac76ec733ee2b8c01ab036950'
URL_BASE = 'https://api.stlouisfed.org/fred/series/observations'
TODAY = pd.Timestamp.today().date()

# Create function to get raw json return from FRED database
def get_series_json(series_id, start, end, api_key=API_KEY, file_type='json', url_base=URL_BASE):
    url = f'{url_base}?series_id={series_id}'
    if start is not None:
        start = pd.to_datetime(start, errors='raise')
        url += '&observation_start=' + start.strftime('%Y-%m-%d')
    else:
        start = TODAY - dt.timedelta(years=1)
        url += '&observation_start=' + start.strftime('%Y-%m-%d')
    if end is not None:
        end = pd.to_datetime(end, errors='raise')
        url += '&observation_end=' + end.strftime('%Y-%m-%d')
    
    url += f'&api_key={api_key}&file_type={file_type}'
    try:
        resp = requests.get(url)
        resp.raise_for_status()  # Raise exception if invalid response
        return resp
    except Exception as e:
        errmsg = resp.json()['error_message'].replace('series', f'series {series_id}')
        print(f'Error: {resp.status_code}\n{errmsg}')
        return None
    
# Create function to transform valid json response from FRED into a dataframe
def transform_series_json(resp, series_id):
    resp = resp.json()
    obs = pd.DataFrame(resp.pop('observations'))[['date', 'value']]
    obs['date'] = pd.to_datetime(obs['date'])
    obs.set_index('date', inplace=True)
    meta = pd.DataFrame({
        series_id: {'observation_start': resp['observation_start'],
            'observation_end': resp['observation_end'],
            'count': resp['count'],
            'actual_start': obs.index.min().date(),
            'actual_end': obs.index.max().date(),
            'NaN count': obs[obs.value == '.'].count().value}})
    
    obs.loc[obs.value == '.'] = np.nan
    obs.columns = [series_id]
    obs[series_id] = obs[series_id].astype(float, errors='raise')
    return obs, meta
    
# Create function to fill missing values in FRED series datafame
def fill_series_na(df):
    df.fillna(method='ffill', inplace=True)  # Fill missing values with last observation
    df.fillna(method='bfill', inplace=True)  # Then, fill with next observation
    return df

# Create a function to get a time series from FRED and return a clean dataframe
def get_series(series_id, start, end, api_key=API_KEY, file_type='json', fill_na=None, multi=None):
    fill_na = True if fill_na is None else fill_na  # Default
    multi = False if multi is None else multi  # Default
    try:
        resp = get_series_json(series_id=series_id, start=start, end=end, api_key=api_key, file_type=file_type)
        df, meta = transform_series_json(resp, series_id=series_id)
        df = fill_series_na(df) if fill_na else df
    except Exception as e:
        print(f'Error: {e}')
        return None
    print(f'Downloaded {series_id} \n{meta.to_markdown()}') if not multi else None
    return df, meta

# def get_multiple_series()



series = 'DEXUSUK'
start = '1983-01-01'
end = '2022-12-31'

df, meta = get_series(series_id=series, start=TODAY, end=end, fill_na=False)

In [4]:
meta

,DEXUSUK
NaN count,395
actual_end,2022-12-30
actual_start,1983-01-03
count,10435
observation_end,2022-12-31
observation_start,1983-01-01


In [24]:
# Define function to use the get_series function to get the data for a list of series
# and combine them into one dataframe

def get_multiple_series(series_list, start, end, fill_na=True):
    df = pd.DataFrame()
    metalist = []
    for series in series_list:
        try:
            df_series, meta = get_series(series_id=series, start=start, end=end, fill_na=fill_na)
            df = pd.concat([df, df_series], axis=1)
            metalist.append(meta)
        except Exception as e:
            print(e)
    return df, metalist

series_list = ['DEXUSUK', 'DEXCAUS', 'DEXCHUS', 'DEXJPUS', 'DEXINUS', 'DEXSFUS']
start = '1983-01-01'
end = '2022-12-31'

list_df, meta = get_multiple_series(series_list, start=start, end=end)
print(list_df)
pd.concat(meta, axis=1).T

            DEXUSUK  DEXCAUS  DEXCHUS  DEXJPUS  DEXINUS  DEXSFUS
date                                                            
1983-01-03   1.6235   1.2300   1.9275   232.00     9.62   1.0695
1983-01-04   1.6210   1.2298   1.9140   229.80     9.64   1.0667
1983-01-05   1.6210   1.2297   1.9140   229.10     9.64   1.0684
1983-01-06   1.6065   1.2313   1.9044   229.80     9.70   1.0712
1983-01-07   1.6100   1.2267   1.9044   229.10     9.73   1.0712
...             ...      ...      ...      ...      ...      ...
2022-12-26   1.2054   1.3587   6.9880   132.78    82.86  16.9725
2022-12-27   1.2032   1.3504   6.9600   133.43    82.88  17.3030
2022-12-28   1.2034   1.3588   6.9774   134.27    82.74  17.1150
2022-12-29   1.2060   1.3546   6.9625   133.16    82.83  16.8900
2022-12-30   1.2077   1.3532   6.8972   131.81    82.72  16.9950

[10435 rows x 6 columns]


,NaN count,actual_end,actual_start,count,observation_end,observation_start
DEXUSUK,395,2022-12-30,1983-01-03,10435,2022-12-31,1983-01-01
DEXCAUS,395,2022-12-30,1983-01-03,10435,2022-12-31,1983-01-01
DEXCHUS,456,2022-12-30,1983-01-03,10435,2022-12-31,1983-01-01
DEXJPUS,395,2022-12-30,1983-01-03,10435,2022-12-31,1983-01-01
DEXINUS,403,2022-12-30,1983-01-03,10435,2022-12-31,1983-01-01
DEXSFUS,0,2022-12-30,1983-01-03,10435,2022-12-31,1983-01-01


In [25]:
# Orient each currency pair so that USD is the quote currency (denominator)
# This is so that the charts will make more intuitive sense as an increase will 
# mean that the foreign currencies are strengthening against the USD and vice versa
aligned_df = list_df.copy()
for i in aligned_df:
    if i[-2:] == 'US':
        aligned_df[i] = aligned_df[i].rdiv(1)
aligned_df.head()

,DEXUSUK,DEXCAUS,DEXCHUS,DEXJPUS,DEXINUS,DEXSFUS
date,,,,,,
1983-01-03,1.6235,0.813008,0.518807,0.004310,0.103950,0.935016
1983-01-04,1.6210,0.813140,0.522466,0.004352,0.103734,0.937471
1983-01-05,1.6210,0.813206,0.522466,0.004365,0.103734,0.935979
1983-01-06,1.6065,0.812150,0.525100,0.004352,0.103093,0.933532
1983-01-07,1.6100,0.815195,0.525100,0.004365,0.102775,0.933532


In [26]:
# Get cumulative returns for each currency for comparison
aligned_diff = aligned_df.pct_change().dropna().add(1).cumprod()
aligned_ccys = ['GBP', 'CAD', 'CHF', 'JPY', 'INR', 'ZAR']
aligned_diff.columns = aligned_ccys
aligned_diff.head()

,GBP,CAD,CHF,JPY,INR,ZAR
date,,,,,,
1983-01-04,0.998460,1.000163,1.007053,1.009574,0.997925,1.002625
1983-01-05,0.998460,1.000244,1.007053,1.012658,0.997925,1.001030
1983-01-06,0.989529,0.998944,1.012130,1.009574,0.991753,0.998413
1983-01-07,0.991685,1.002690,1.012130,1.012658,0.988695,0.998413
1983-01-10,0.979058,1.003836,1.012130,1.023153,0.989712,1.004791


In [27]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Get min/max values for scaling
max_x = aligned_diff.index.max()
min_x = aligned_diff.index.min()
max_y = aligned_diff.max().max()
min_y = aligned_diff.min().min()

# Set the hovertemplate to percent for more readable hover labels
ht = '%{y:,.1%}'

layout = {
    'title': '<b>Currency 40-Year Cumulative Percentage Change</b><br><sup><i>(1983-2022)</i></sup>',
    'width': 2400,
    'height': 1200,
    'template': 'seaborn',
    'hovermode': 'x unified',
}

fig = make_subplots(rows=2, cols=3, shared_xaxes=True, vertical_spacing=0.05, horizontal_spacing=0.02,
                    subplot_titles=([ccy for ccy in aligned_diff]), shared_yaxes=True, 
                    x_title='Date 1983-2022', y_title='Cumulative Foreign Currency Rate Percentage Change VS US Dollar')

for i, ccy in enumerate(aligned_diff):
    trace = go.Scatter(x=aligned_diff.index, y=aligned_diff[ccy], mode='lines', name=ccy, hovertemplate=ht)
    if i // 3 < 1:
        fig.add_trace(trace, row=1, col=i+1)
    else:
        fig.add_trace(trace, row=2, col=i-2)

fig.update_layout(layout)
for i in fig.layout.annotations[-2:]:
    i.font.size = 20  # Bit of a hacky way to change the font size of the x/y labels, but it works

fig.show()

: 

In [5]:
data = aligned_traces
ht = '%{y} %{y:,.1%}'
layout = {
    'title': 'USD/GBP Exchange Rate',
    'xaxis_title': 'Date',
    'yaxis_title': 'USD/GBP Rate',
    'width': 2400,
    'height': 1200,
    'hovermode': 'x unified',
    'hoverlabel_align': 'right',
    'template': 'seaborn',
    'xaxis': {
        'rangeslider': {
            'visible': True,
        },
    }
}

fig = go.Figure(data=data, layout=layout)
fig.show()

NameError: name 'aligned_traces' is not defined

In [ ]:
fig.__dict__['_layout']['hoverlabel']

In [ ]:
import plotly.graph_objects as go

trace = go.Scatter(x=df.index, y=df['DEXUSUK'], mode='lines')

data = [trace]
layout = {
    'title': 'USD/GBP Exchange Rate',
    'xaxis_title': 'Date',
    'yaxis_title': 'USD/GBP Rate',
    'width': 1800,
    'height': 1000,
    'hovermode': 'x unified',
    'hoverlabel_align': 'right',
    'template': 'seaborn',
    'xaxis': {
        'rangeslider': {
            'visible': True,
        },
    }
}

fig = go.Figure(data=data, layout=layout)
fig.show()

In [ ]:
df.resample('W').mean()

In [ ]:
# Calculate seasonal decomposition on the DEXUSUK time series
from statsmodels.tsa.seasonal import seasonal_decompose

# decomp = seasonal_decompose(df)
decomp = seasonal_decompose(df.resample('W').mean(), extrapolate_trend='freq')

# Show decomposition plots
from plotly.subplots import make_subplots

fig = make_subplots(rows=4, cols=1, subplot_titles=['Observed', 'Trend', 'Seasonal', 'Residuals'])

fig.add_trace(go.Scatter(x=decomp.observed.index, y=decomp.observed.values, name='Observed'), row=1, col=1)
fig.add_trace(go.Scatter(x=decomp.trend.index, y=decomp.trend.values, name='Trend'), row=2, col=1)
fig.add_trace(go.Scatter(x=decomp.seasonal.index, y=decomp.seasonal.values, name='Seasonal'), row=3, col=1)
fig.add_trace(go.Scatter(x=decomp.resid.index, y=decomp.resid.values, name='Residuals'), row=4, col=1)

fig.update_layout(width=1800, height=1000, title='Seasonal Decomposition Plot', template='seaborn')

fig.show()

In [ ]:
from pmdarima import auto_arima

arimafit = auto_arima(df['DEXUSUK'].dropna(), trace=True)
# arimafit = auto_arima(df['DEXUSUK'].resample('W').mean().dropna(), trace=True)
arimafit.summary()

In [ ]:
train = df['DEXUSUK'].iloc[:-90]
test = df['DEXUSUK'].iloc[-90:]

In [ ]:
from statsmodels.tsa.arima.model import ARIMA

mod = ARIMA(train, order=(1,1,0), freq='B')
res = mod.fit()
res.summary()

In [ ]:
from statsmodels.tsa.stattools import adfuller                                                                                                                                                                              


# Evaluate the stationarity of time series data using Augmented Dickey-Fuller and print results
def adf_test(ts):
    df = adfuller(ts, autolag='AIC')
    results = pd.DataFrame(df[:4], columns=['Results '], dtype=object)
    results.index=['Test Statistic', 'p-value', 'Num Lags', 'Num Observations']
    print('Results of Augmented Dickey-Fuller Test:\n\n', results)
    if df[1] <= 0.05:
        print('\nStrong evidence against the null hypothesis')
        print('Reject the null hypothesis')
        print('Data may have no unit root and is stationary')
    else:
        print('\nWeak evidence against the null hypothesis')
        print('Fail to reject the null hypothesis')
        print('Data may have a unit root and is non-stationary')

# Run the ADF test on the raw time series data
adf_test(df)

# Plot 30-day rolling mean
df.rolling(window=30).mean().plot()

In [ ]:
res.forecast(10).value_counts()

In [ ]:
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_predict

def arima_forecast_plot(model, train, test, title, ylabel, xlabel, start=len(train), end=len(train)+len(test), figsize=(16, 8), plt_ext=-270):
    fig, ax = plt.subplots(figsize=figsize)
    ax.set(title=title, xlabel=xlabel, ylabel=ylabel)
    train.plot(ax=ax)
    test.plot(ax=ax, color='teal')
    plot_predict(model, start=start, end=end, ax=ax, alpha=0.05)
    
    ax.legend(['Training Data', 'Testing Data', 'Predictions'])
    plt_range_start = train.index[plt_ext]
    plt_range_end = test.index[-1]
    plt_max = max(train.iloc[plt_ext:].max(), test.max()) * 1.1
    plt_min = min(train.iloc[plt_ext:].min(), test.min()) * 0.9
    plt.xlim([plt_range_start, plt_range_end])
    plt.ylim([plt_min, plt_max])
    plt.show()
arima_forecast_plot(res, train, test, title='GBP/USD FX Rate Forecasts (90 Days)', ylabel='FX Rate', xlabel='Date')

In [ ]:
plot_predict(res, len(train), len(test) + len(train)).__class__

In [ ]:
trace = go.Scatter(x=df.index, y=df['DEXUSUK'], mode='lines')
data = [trace]
layout = {
    'title': 'USD/GBP Exchange Rate',
    'xaxis_title': 'Date',
    'yaxis_title': 'USD/GBP Rate',
    'width': 1800,
    'height': 1000,
    'hovermode': 'x unified',
    'template': 'seaborn',
    'xaxis': {
        'rangeslider': {
            'visible': True,
        },
    }
}

fig = go.Figure(data=data, layout=layout)
fig.show()